# Photon response

This notebook builds up on the output of the [HGCDigiTester](https://github.com/PFCal-dev/HGCElectronicsValidation/blob/master/src/HGCDigiTester.cc) tool.
The reconstructed hits are aggregated into showers and different levels of corrections are applied to build the energy estimator of the shower:

* dE/dx weights
* charge collection, light yield and geometry corrections
* sums per sensor type and section

The summaries just need to be created once as they take a bit of time to process:

* In case you need to produce the summaries, please connect first to the k8s spark cluster and set spark.driver.maxResultSize=0 (for unlimited output). Alternatively you can run in multithread locally. It seems to be more efficient for the ntuples of 26/Nov/2020
* In case the summaries are already available, one can jump directly to the plotting part of the notebook.

In [1]:
import os
import ROOT
import pandas as pd
import PyRDF
import numpy as np

baseDir='/eos/cms/store/cmst3/user/psilva/HGCAL/emResponse'
flist=[os.path.join(baseDir,f) for f in os.listdir(baseDir) if '.root' in f]

Welcome to JupyROOT 6.20/06


# 1. Data preparation

## Build the dataframes for analysis

This step will take a bit of time as it requires to analyze all the hits and apply the corrections/sums defined above.

In [2]:
runLocal=True

#local
if runLocal:
    ROOT.ROOT.EnableImplicitMT()
    PyRDF.use('local')

#spark configuration
#cf https://spark.apache.org/docs/latest/configuration#available-properties for details
#maxResultSize is set to unlimited however it seems to work only when set to 0 at connection time in the form
else:
    spark_conf={'npartitions':12,
                'spark.driver.maxResultSize':0, 
                }
    PyRDF.use('spark',conf=spark_conf)

#include helper header
PyRDF.include_headers("PhotonResponse_helpers.h")

def getSummaryFrom(url,ran=None):
    
    """convert trees to dataframes and pre-select them for calibration"""
    
    print('Starting analysis of',url)
    if runLocal:
        files=[url]
    else:
        files = ['root://eosuser.cern.ch/{}'.format(url)]
    rdf = PyRDF.RDataFrame("ana/hits", files)
    
    #filter event range if required
    if ran:
        rdf=rdf.Filter('event>={} && event<={}'.format(*ran))

    #filter the hits (at least 1 simulated MIP deposit)    
    rdf=rdf.Filter('mipsim>=1.0')
    
    #helper variables
    rdf=rdf.Define('side','z<0 ? -1 : 1')
    for c in ['miprec','avgmiprec']:
        new_c='{}_dedx'.format(c)
        mpvest='true' if c=='miprec' else 'false'
        rdf=rdf.Define(new_c,          'scaledHit(layer,{},cce,true,false)'.format(c)) \
               .Define(new_c+'_cce',   'scaledHit(layer,{},cce,true,true)'.format(c)) \
               .Define(new_c+'_cce_sw','swCompensatedHit({}_cce,thick,isSci,{})'.format(new_c,mpvest))
    
    #all done
    df=pd.DataFrame( rdf.AsNumpy() )
    print('\tFilled with',df.shape[0],'hits to analyze from',len(df['event'].unique()),'events')
        
    return df

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/cvmfs/sft.cern.ch/lcg/releases/spark/2.4.5-cern1-f7679/x86_64-centos7-gcc8-opt/jars/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/cvmfs/sft.cern.ch/lcg/releases/hadoop/2.7.5.1-79196/x86_64-centos7-gcc8-opt/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
21/01/13 15:28:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
#loop over available files
rewrite=False
for url in flist:
    new_url = url.replace('.root','.h5')
    if os.path.isfile(new_url) and not rewrite: continue
    try:
        ran=[0,5000] if 'Flat' in url else None
        hits=getSummaryFrom(url,ran=ran)
        hits.to_hdf(new_url,key='hits',mode='w')
        print(hits[['layer','gvz','gvradius','genergy']].describe())
    except Exception as e:
        print(e)

## Build the shower energy estimators

To sum the total energy per shower we index the data using (event,side), assuming one particle in each endcap, and sum up the different energy estimators.

In [2]:
categs=['CEE_Si120',      'CEE_Si200',       'CEE_Si300',
        'CEHfine_Si120',  'CEHfine_Si200',   'CEHfine_Si300',   'CEHfine_Sci',
        'CEHcoarse_Si120','CEHcoarse_Si200', 'CEHcoarse_Si300', 'CEHcoarse_Sci',        
        'last']

def getMaskForCateg(df,categ):

    """defines a mask for different categories"""

    mask=(df['layer']<=28)
    if 'CEHfine' in categ:   mask=(df['layer']>28) & (df['layer']<41)
    if 'CEHcoarse' in categ: mask=(df['layer']>40)
    if 'last' in categ:      mask=(df['layer']>48)
    
    if 'Sci' in categ:
        mask &= (df['isSci']==1)
    elif 'Si' in categ:
        if '120' in categ: mask &= (df['thick']==0)
        if '200' in categ: mask &= (df['thick']==1)
        if '300' in categ: mask &= (df['thick']==2)
            
    return mask        
        
def getShowerSummaryForCalibration(df):
    
    """sums up the energy estimators per shower"""
    
    #index the showers by event number and side
    df_indexed=df.set_index(['event','side'])
    level=[0,1]
    
    #start to describe the contents of the summary
    shower_desc = { 'genergy'           : df_indexed.mean(level=level)['genergy'],
                    'geta'              : np.abs(df_indexed.mean(level=level)['geta']),
                    'gvradius'          : df_indexed.mean(level=level)['gvradius'],
                    'gvz'               : df_indexed.mean(level=level)['gvz'],
                    'thick'             : df_indexed.mean(level=level)['thick'],
                    'isSci'             : df_indexed.mean(level=level)['isSci'],
                    'totalSat'          : df_indexed.sum(level=level)['isSat'],
                  }

    #define the columns with the values ready to be summed for the reconstruction of the energy
    for c in ['miprec','avgmiprec']:
                
        for cat in categs:
            mask=getMaskForCateg(df_indexed,cat)

            df_indexed['{}_dedx_{}'.format(c,cat)]        = np.where(mask,df_indexed['{}_dedx'.format(c)],0.)
            shower_desc['total_{}_dedx_{}'.format(c,cat)] = df_indexed.sum(level=level)['{}_dedx_{}'.format(c,cat)]       

            df_indexed['{}_dedx_cce_{}'.format(c,cat)]        = np.where(mask,df_indexed['{}_dedx_cce'.format(c)],0.)
            shower_desc['total_{}_dedx_cce_{}'.format(c,cat)] = df_indexed.sum(level=level)['{}_dedx_cce_{}'.format(c,cat)] 
            
            df_indexed['{}_dedx_cce_sw_{}'.format(c,cat)]        = np.where(mask,df_indexed['{}_dedx_cce_sw'.format(c)],0.)
            shower_desc['total_{}_dedx_cce_sw_{}'.format(c,cat)] = df_indexed.sum(level=level)['{}_dedx_cce_sw_{}'.format(c,cat)] 
            
        shower_desc['total_{}_dedx'.format(c)] = df_indexed.sum(level=level)['{}_dedx'.format(c)]       
        shower_desc['total_{}_dedx_cce'.format(c)] = df_indexed.sum(level=level)['{}_dedx_cce'.format(c)] 
        shower_desc['total_{}_dedx_cce_sw'.format(c)] = df_indexed.sum(level=level)['{}_dedx_cce_sw'.format(c)] 
            
    #build the shower summary and filter in case there are saturated hits
    shower_data = pd.DataFrame(shower_desc)
    mask = (shower_data['totalSat']==0)
    shower_data=shower_data[mask]
    shower_data.drop(columns=['totalSat'],inplace=True)
    shower_data.reset_index(inplace=True)

    return shower_data

In [ ]:
#loop over the hit summaries
hit_flist=[os.path.join(baseDir,f) for f in os.listdir(baseDir) if '.h5' in f]
for url in hit_flist:
    
    if not 'Flat' in url: continue
    
    print('Starting with',url)
    
    #get hits and build the shower data summary
    df = pd.read_hdf(url,key='hits',mode='a')
    if 'Flat' in url:
        print('\tLimiting to 5k events (10k photons)')
        df=df[df['event']<5000]
    
    try:
        showers_df=pd.read_hdf(url,key='showers',mode='a')
        print(showers_df.shape)
        print('Shower summaries already available, moving to next')     
        continue
    except:
        pass
    
    print('\tLayers in range',df['layer'].min(),df['layer'].max())
    
    showers_df = getShowerSummaryForCalibration(df)
    print('\tSelected',showers_df.shape[0],'showers for calibration')

    showers_df.to_hdf(url,key='showers',mode='a')
    print('\tSummary @',url)

Starting with /eos/cms/store/cmst3/user/psilva/HGCAL/emResponse/FlatRandomPtGunProducer_16x_vanilla_20201112.h5
	Limiting to 5k events (10k photons)
	Layers in range 1 50


# 2. Data analysis

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
import matplotlib as mpl
mpl.rcParams['font.size'] = 14

sampleTitles={'CloseByParticleGunProducer_16x_vanilla_20201114.h5'      : 'Non-pointing $\gamma$ vanilla',
              'CloseByParticleGunProducer_aged3iab_20201117.h5'         : 'Non-pointing $\gamma$ 3ab$^{-1}$',
              'CloseByParticleGunProducer_realisticStartup_20201123.h5' : 'Non-pointing $\gamma$ realistic startup',
              'CloseByParticleGunProducer_aged3iab_20201127.h5'         : 'Non-pointing $\gamma$, 3ab$^{-1}$',
              'CloseByParticleGunProducer_realisticStartup_20201127.h5' : 'Non-pointing $\gamma$, realistic startup',
              'FlatRandomPtGunProducer_16x_vanilla_20201112.h5'         : 'Pointing $\gamma$ vanilla',
              'FlatRandomPtGunProducer_aged3iab_20201022.h5'            : 'Pointing $\gamma$ 3ab$^{-1}$',
              'FlatRandomPtGunProducer_aged3iab_20201023.h5'            : 'Pointing $\gamma$ 3ab$^{-1}$',
              'FlatRandomPtGunProducer_realisticStartup_20201110.h5'    : 'Pointing $\gamma$ realistic startup',
}

## Linearity between injected and reconstructed charge

Plots the sim vs rec charge and the distribution of the response

In [ ]:
files=[f for f in os.listdir(baseDir) if '.h5' in f]
resp_bins=np.linspace(0.5,1.5,100)
for f in files:
    url=os.path.join(baseDir,f)
    if not f in sampleTitles.keys(): continue
    df=pd.read_hdf(url,key='hits').iloc[:100000] #no need to use all of them for this check
    
    nrows=4 if 'CloseByParticleGun' in f else 3
    fig,ax=plt.subplots(1,nrows,figsize=(nrows*5.5,5))
    fig2,ax2=plt.subplots(1,nrows,figsize=(nrows*5.5,5))

    i=0
    for title,mask in [(r'120 $\mu$m',(df['thick']==0) & (df['isSat']==0)),
                       (r'200 $\mu$m',(df['thick']==1) & (df['isSat']==0)),
                       (r'300 $\mu$m',(df['thick']==2) & (df['isSat']==0)),
                       ('Scintillator',(df['isSci']==1) & (df['isSat']==0))]:         

        ax[i].scatter(df[mask]['mipsim'],df[mask]['miprec'],label='before CCE')
        ax[i].scatter(df[mask]['mipsim'],df[mask]['miprec']/df[mask]['cce'],label='after CCE')
        ax[i].set_xlabel('Sim MIPs')
        ax[i].legend(loc='upper left',title=title)
        ax[i].grid()
        
        ax2[i].hist(df[mask]['miprec']/df[mask]['mipsim'],histtype='step',linewidth=2,bins=resp_bins,label='before corr.')
        ax2[i].hist( (df[mask]['miprec']/df[mask]['cce'])/df[mask]['mipsim'],histtype='step',linewidth=2,bins=resp_bins,label='after corr.')
        mask &= (df['isTOT']==1)
        ax2[i].hist( (df[mask]['miprec']/df[mask]['cce'])/df[mask]['mipsim'],histtype='step',linewidth=2,bins=resp_bins,label='after corr. (TOT)')
    
        ax2[i].set_xlabel('Rec/Sim')
        ax2[i].legend(loc='upper left',title=title)
        ax2[i].grid()

        if i==0:
            ax[i].set_ylabel('Rec MIPs')
            ax2[i].set_ylabel('Hits')
            ax[i].text(0.05,1.05,r'CMS Simulation {}'.format(sampleTitles[f]),transform=ax[i].transAxes,fontsize=16)
            ax2[i].text(0.05,1.05,r'CMS Simulation {}'.format(sampleTitles[f]),transform=ax2[i].transAxes,fontsize=16)

        i=i+1
        if i>nrows-1 : break 

    plt.show()

## Typical energy spectrum from the simulated hits

Distribution of charge in the pads generated by photons originated from (0,0,0)

In [ ]:
df=pd.read_hdf('/eos/cms/store/cmst3/user/psilva/HGCAL/emResponse/FlatRandomPtGunProducer_16x_vanilla_20201112.h5',key='hits')

mask=(df['thick']==2)
mipsim=df[mask]['mipsim']
gpt=df[mask]['genergy']/np.cosh(df[mask]['geta'])
mipfC=3.6087

bins=np.percentile(mipsim,[x*5 for x in range(21)], axis=0)
sel_p=[10,50,90]
sel_quantiles=np.percentile(mipsim,sel_p, axis=0)
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.set_ylabel(r'dN$_{hits}$/dMIPs')
ax1.set_xlabel(r'MIPs')
ax1.set_xscale('log')
ax1.hist(mipsim,         bins=bins, density=True, histtype='step', lw=2, label=r'10<p$_{T}$<100 GeV')
ax1.hist(mipsim[gpt<20], bins=bins, density=True, histtype='step', lw=2, label=r'p$_{T}$<25 GeV')
ax1.hist(mipsim[gpt>90], bins=bins, density=True, histtype='step', lw=2, label=r'p$_{T}$>90 GeV')
ax1.legend()
ax1.set_yscale('log')
for i in range(3):
    ax1.plot([sel_quantiles[i],sel_quantiles[i]],[0,0.2],ls='--',color='gray')
    print('quantile ',sel_p[i],':',sel_quantiles[i],'MIPs = ',sel_quantiles[i]*mipfC,'fC')

ax1.grid()

ax2 = ax1.twiny()
ax2.set_xscale('log')
new_tick_locations = np.array([1/mipfC,5/mipfC,10/mipfC,50/mipfC,100/mipfC,500/mipfC,1000/mipfC])
def tick_function(X):
    V = X*mipfC
    return ["%.0f" % z for z in V]
ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
ax2.set_xlabel(r"q [fC]")

plt.show()

## Photon energy response

In [ ]:
varTitles={'geta':'|\eta|',
           'gvz' : 'Vertex~z~[cm]',
           'gvradius' : 'Vertex~radius~[cm]',
           'genergy':'E_{gen}/GeV'}

def showResponseProfile(summary,
                        en_est=['total_miprec_dedx','total_miprec_dedx_cce'],
                        en_est_label=['dE/dx','dE/dx+corr.'],
                        en_est_fit=[False,True],
                        xvar='geta',
                        ytitle='Reconstructed/Generated',
                        title='',
                        yran=None):
    
    fig,ax=plt.subplots(figsize=(6,5))
    
    x = abs(summary[xvar])
    xvar_title=varTitles[xvar]
    for i,iy in enumerate(en_est):
        
        y = 1e-3*summary[iy]/summary['genergy']
        plt.scatter(x,y,label=en_est_label[i])

        #fit the average reponse if required
        if not en_est_fit[i]: continue
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        yexp = slope*x+intercept
        plt.plot(x,yexp,'r',label=r'R={:3.4f}$\cdot {}$+{:3.3f}'.format(slope,xvar_title,intercept))
    
    plt.xlabel(r'${}$'.format(xvar_title))
    plt.ylabel(ytitle)
    if yran: plt.ylim(*yran)
    plt.grid()
    plt.legend(loc='lower right')
    plt.text(0.05,1.05,r'CMS Simulation {}'.format(title),transform=ax.transAxes,fontsize=16)
    plt.tight_layout()
    plt.show()   


def showResponseByThickness(summary,
                            en_est='total_miprec_dedx_cce',
                            xtitle='Reconstructed/Generated',
                            title='',
                            xran=(1.0,1.4)):
    
    fig,ax=plt.subplots(figsize=(6,5))
    bins=np.linspace(xran[0],xran[1],50)
    for label,mask in [('120$\mu$m',abs(summary['thick']-0)<0.01),
                       ('200$\mu$m',abs(summary['thick']-1)<0.01),
                       ('300$\mu$m',abs(summary['thick']-2)<0.01),
                      ]:

        x = 1e-3*summary[mask][en_est]/summary[mask]['genergy']
        plt.hist(x,bins=bins,label=label,histtype='step',linewidth=2)
    plt.xlabel(xtitle)
    plt.grid()
    plt.legend(loc='best')
    plt.text(0.05,1.05,r'CMS Simulation {}'.format(title),transform=ax.transAxes,fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
for sample in ['FlatRandomPtGunProducer_aged3iab_20201022.h5',
               #'FlatRandomPtGunProducer_aged3iab_20201023.h5',
               #'FlatRandomPtGunProducer_realisticStartup_20201110.h5', 
              ]:
    
    df=pd.read_hdf(os.path.join(baseDir,sample),key='showers')
    df=df[(df['geta']>1.6) & (df['geta']<2.9)]

    #showResponseProfile(df,title=sampleTitles[sample],xvar='gvz')
    #showResponseByThickness(df,en_est='total_miprec_dedx_cce',title=sampleTitles[sample])

    #showResponseProfile(df,en_est=['total_avgmiprec_dedx','total_avgmiprec_dedx_cce'],title=sampleTitles[sample])
    #showResponseByThickness(df,en_est='total_avgmiprec_dedx_cce',title=sampleTitles[sample],xran=(0.6,1.0))
    
    showResponseProfile(df,en_est=['total_avgmiprec_dedx','total_avgmiprec_dedx_cce_sw'],title=sampleTitles[sample])
    showResponseByThickness(df,en_est='total_avgmiprec_dedx_cce_sw',title=sampleTitles[sample],xran=(0.6,1.0))

## Local e.m. corrections
We correct the residual difference in response for the three thicknesses by minimizing $E = \sum_{i \in \{120,200,300\} } E_i w_i$. The CCE correction is also applied.

In [ ]:
#HGCAL envelope comes from here https://edms.cern.ch/ui/#!master/navigator/document?P:1467919020:100561389:subDocs
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

hgcal_envelope_pts=0.1*np.array([[2965,260.0],[2965,1243.7],[3185,1364.7],[3185,1589.4],[3837.0,1816],[4522.0,2725],[5265,2725],[5265,900],[5265,900],[5265,360.0],[4430,360.0],[4430,260.0],[2965,260.0]])
hgcal_envelope = Path(hgcal_envelope_pts)

rm=2*30
hgcal_fid_pts=0.1*np.array([[2965,260.0+rm],[2965,1243.7-rm],[3185,1364.7-rm],[3185,1589.4-rm],[3837.0,1816-rm],[4522.0,2725-rm],[5265,2725-rm],
                            [5265,900+rm],[5265,900+rm],[5265,360.0+rm],[4430,360.0+rm],[4430,260.0+rm],[2965,260.0+rm]])
hgcal_fid = Path(hgcal_fid_pts)


def showHGCALEnvelope(pts=None):
    fig, ax = plt.subplots()
    patch = patches.PathPatch(hgcal_envelope, facecolor='orange', alpha=0.25, lw=3)
    ax.add_patch(patch)
    if not pts is None:
        ax.scatter(pts[:,0],pts[:,1])
    ax.set_xlim(250, 600)
    ax.set_ylim(0, 300)
    ax.set_xlabel('z [cm]')
    ax.set_ylabel('r [cm]')
    plt.grid()
    plt.show()
 
#CMSSW acceptance https://hgcal.web.cern.ch/HitCalibration/WorkingExample/
pts=np.array([[321,55],[321,90],[321,135],[430,80],[430,180],[362.52,50],[362.52,90],[362.52,135.0]])
showHGCALEnvelope(pts)

In [ ]:
import statsmodels.api as sm

categs=['CEE_Si120',      'CEE_Si200',       'CEE_Si300',
        'CEHfine_Si120',  'CEHfine_Si200',   'CEHfine_Si300',   'CEHfine_Sci',
        'CEHcoarse_Si120','CEHcoarse_Si200', 'CEHcoarse_Si300', 'CEHcoarse_Sci']

def selectForCalibration(sample,enest,pointing=False,silent=False):
    #open file
    url=os.path.join(baseDir,sample)
    df=pd.read_hdf(url,key='showers')

    #no leakage
    leak_mask=~(df['total_{}_dedx_cce_last'.format(enest)]/df['total_{}_dedx_cce'.format(enest)]>0.01)
    df=df[leak_mask]
    
    #hgcal fiducial pointing / non-pointing
    if pointing:
        mask=(df['geta']>1.6) & (df['geta']<2.9)
        df=df[mask]
    else:
        zr_pts=df[['gvz','gvradius']].to_numpy()
        mask=hgcal_fid.contains_points(zr_pts)
        df=df[mask]
        zr_pts=df[['gvz','gvradius']].to_numpy()
        if not silent:
            showHGCALEnvelope(zr_pts)

    return df
    
def runLocalEMRegression(sample,enest='miprec'):

    df=selectForCalibration(sample,enest)
    
    Esums=['total_{}_dedx_cce_{}'.format(enest,c) for c in categs]
    X = df[Esums]
    y = df['genergy']
    print(y.min(),y.max())

    #ordinary least square method https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html
    ols     = sm.OLS(y, X)
    results = ols.fit()

    new_enest='total_{}_dedx_cce_localem'.format(enest)
    df[new_enest]=1e3*results.predict() #keep in MeV

    print('Linear regression results')
    print(r'Accuracy (R^2):',results.rsquared)
    for ip in range(len(results.params)):
        print('{:15s} {:3.3f} +/- {:3.3f}'.format(categs[ip],1e3*results.params[ip],1e3*results.bse[ip]))

    for xvar in ['gvz','gvradius','genergy']:
        showResponseProfile(df,
                            en_est=['total_{}_dedx'.format(enest),new_enest],
                            title=sampleTitles[sample],
                            xvar=xvar)

    return results
    
enest='avgmiprec'
#tag='CloseByParticleGunProducer_16x_vanilla_20201114.h5'
tag='CloseByParticleGunProducer_realisticStartup_20201127.h5'
emlocalcorr=runLocalEMRegression(tag,enest)

In [ ]:
for sample in sampleTitles:

    df=selectForCalibration(sample,enest,pointing=True if 'Flat' in sample else False)
    Esums=['total_{}_dedx_cce_{}'.format(enest,c) for c in categs]
    X = df[Esums]
    y = df['genergy']
    
    new_enest='total_{}_dedx_cce_localem'.format(enest)
    df[new_enest]=1e3*emlocalcorr.predict(X) #keep in MeV

    showResponseProfile(df,
                        en_est=['total_{}_dedx'.format(enest),new_enest],
                        title=sampleTitles[sample],
                        xvar='gvradius',
#                        xvar='genergy' if 'CloseBy' in sample else 'geta',
                        yran=(0.4,1.4))

## Energy resolution maps

We divide a sample in equal statistics in z and for each z in energy. For each category the effective resolution and the bias is computed.
The results are interpolated creating a smooth map of the e.m. resolution in HGCAL

In [ ]:
def getCentralAndUncertainty(x):
    
    """
    estimates the median and the quantiles for a 68%CI around it
    returns the median, median uncertainty, effective width and the unc. on eff. width
    the uncertainties are compued using a gaussian approximation
    (a la ROOT https://root.cern.ch/doc/master/TH1_8cxx_source.html#l07467)
    """
    
    p=np.percentile(x, [16,50,84])
    sigma=np.std(x,axis=0)
    n=x.shape[0]
    
    #median and uncertainty on median
    r=p[1]
    unc_r=1.253*sigma/np.sqrt(n)
    
    #eff. width and uncertainty on it 
    wid_r=0.5*(p[2]-p[0])
    unc_wid_r=sigma/np.sqrt(2*n)
    
    return [r,unc_r,wid_r,unc_wid_r]

def buildCalibrationSummary(sample,enest,xvar='gvz',yvar='genergy',pointing=False):
    """
    slices the data in two variables ensuring equal statistics in each category
    then computes the bias and the resolution in each category
    returns an array of (x,ex,y,ey,bias,ebias,resol,eresol)
    """

    #prepare the sample and evaluate the energy estimator
    df=selectForCalibration(sample,enest,silent=True,pointing=pointing)    
    Esums=['total_{}_dedx_cce_{}'.format(enest,c) for c in categs]
    X = df[Esums]
    y = df['genergy']
    new_enest='total_{}_dedx_cce_localem'.format(enest)
    df[new_enest]=1e3*emlocalcorr.predict(X) #keep in MeV
    
    #summarize the bias/resolution in different bins
    calibSummary=[]
    xbins=np.percentile(df[xvar], np.linspace(0,100,10))
    for i in range(len(xbins)-1):

        xmask=(df[xvar]>=xbins[i]) & (df[xvar]<xbins[i+1])        
        ybins=np.percentile(df[xmask][yvar],np.linspace(0,100,15))
        for j in range(len(ybins)-1):
        
            ymask=(df[xmask][yvar]>=ybins[j]) & (df[xmask][yvar]<ybins[j+1])
            residuals=df[xmask & ymask][new_enest]/1000.-df[xmask & ymask]['genergy']
            calibSummary.append( [i,j,pointing]
                                 +getCentralAndUncertainty(df[xmask & ymask][xvar])
                                 +getCentralAndUncertainty(df[xmask & ymask][yvar])
                                 +getCentralAndUncertainty(residuals)
                                 )
            
    return pd.DataFrame(calibSummary,
                        columns=['ix','ij','pointing','x','ex','sx','esx','y','ey','sy','esy','r','er','s','es'])

In [ ]:
calibSummaries=[]
i=0
for sample in sampleTitles:
    print('Building calibration summary for',sample)
    
    pointing=True if 'Flat' in sample else False
    xvar='geta' if pointing else 'gvz'
    calibSummaries.append( buildCalibrationSummary(sample,enest,xvar=xvar,pointing=pointing) )
    calibSummaries[-1]['sample']=i
    i+=1
    
calibSummaries=pd.concat(calibSummaries)
fout=os.path.join(baseDir,'calibration.h5')    
calibSummaries.to_hdf(fout,key='calibration',mode='w')
calibSummaries.head()

In [ ]:
i=0
for sample in sampleTitles:
    print(i,sample)
    i=i+1

In [ ]:
def showSummaryResidualsAndResolution(df,title):
    
    fig,ax=plt.subplots(1,2,figsize=(15,5))

    for ix in sorted(df['ix'].unique()):
        mask=(df['ix']==ix)        
        x=df[mask]['y']
        xerr=df[mask]['ey']

        y=df[mask]['r']/x
        yerr=df[mask]['er']/x                 
        ax[0].errorbar(x,y,yerr,xerr,marker='s',ms=10,ls='None',label='<z>={:3.0f} cm'.format(df[mask]['x'].mean()))

        y=df[mask]['s']/x
        yerr=df[mask]['es']/x                 
        ax[1].errorbar(x,y,yerr,xerr,marker='s',ms=10,ls='None')
    
    for i in range(2):
        ax[i].grid()
        ax[i].set_xlabel('E [GeV]',fontsize=16)

    ax[0].legend(ncol=2)
    ax[0].text(0.01,1.05,r'CMS Simulation {}'.format(title),transform=ax[0].transAxes,fontsize=16)
    ax[0].set_ylim(-0.08,0.08)
    ax[1].set_ylim(0,0.22)
    ax[0].set_ylabel(r'$\Delta E/E$',fontsize=16)
    ax[1].set_ylabel(r'$\sigma_E/E$',fontsize=16)
    plt.tight_layout()
    plt.show()


fout=os.path.join(baseDir,'calibration.h5') 
df=pd.read_hdf(fout,key='calibration')

aged=df[df['sample']==3]
showSummaryResidualsAndResolution(aged,sampleTitles['CloseByParticleGunProducer_aged3iab_20201127.h5'])

realistic=df[df['sample']==4]
showSummaryResidualsAndResolution(realistic,sampleTitles['CloseByParticleGunProducer_realisticStartup_20201127.h5'])                                                                    

In [ ]:
from scipy.optimize import curve_fit

def resol_func(x, a, b, c):
    return np.sqrt((a/np.sqrt(x))**2+(b/x)**2+c**2)

def compareResolutions(dfList,titleList):

        fig,ax=plt.subplots(figsize=(6,5))
        fit_results=[]
        
        for i,df in enumerate(dfList):
   
            x=df['y']
            xerr=df['ey']
            bias=df['r']
            y=df['s']/(x+bias)
            yerr=df['es']/(x+bias)                 
            xlin=np.linspace(x.min(),x.max(),100)
                            
            if i==0:
                popt,pcov = curve_fit(lambda x, a,c: resol_func(x, a,0,c),  x, y, sigma=yerr, bounds=[(0.15,0),(0.3,0.05)])
                params=r'$\frac{%3.3f \pm %3.3f}{\sqrt{E}}\oplus'%(popt[0],np.sqrt(pcov[0][0]))
                params+=r'%3.3f \pm %3.3f$'%(popt[1],np.sqrt(pcov[1][1]))                
                ax.plot(xlin,resol_func(xlin,popt[0],0.,popt[1]),'b-')
                ax.errorbar(x,y,yerr,xerr,marker='s',ms=10,ls='None',label='%s\n%s'%(titleList[i],params),color='b')
            else:
                continue
                popt,pcov = curve_fit(resol_func, x, y, sigma=yerr, bounds=[(0.1,0.,0.),(0.3,0.5,0.05)])
                params=r'$\frac{%3.3f \pm %3.3f}{\sqrt{E}}\oplus'%(popt[0],np.sqrt(pcov[0][0]))
                params+=r'\frac{%3.3f \pm %3.3f}{E}\oplus'%(popt[1],np.sqrt(pcov[1][1]))
                params+=r'%3.3f \pm %3.3f$'%(popt[2],np.sqrt(pcov[2][2]))                
                ax.plot(xlin,resol_func(xlin,*popt),'r-')
                ax.errorbar(x,y,yerr,xerr,marker='s',ms=10,ls='None',label='%s\n%s'%(titleList[i],params),color='r')

            fit_results.append( [popt,pcov])
        
        ax.set_xlabel('E [GeV]',fontsize=14)
        ax.set_ylabel(r'$\sigma_E~/~E$  [GeV]',fontsize=14)
        ax.text(0.01,1.05,r'CMS Simulation',transform=ax.transAxes,fontsize=16)
        plt.legend(fontsize=12)
        plt.grid()
        plt.show()
        
compareResolutions([realistic[(realistic['ix']==0) & ((realistic['y']<26) | (realistic['y']>40))],
                    aged[(aged['ix']==0)& ((aged['y']<26) | (aged['y']>40))] ],
                   [sampleTitles['CloseByParticleGunProducer_realisticStartup_20201127.h5'],sampleTitles['CloseByParticleGunProducer_aged3iab_20201127.h5']])

